In [1]:
# パッケージのインストール（初回のみ実行）
using Pkg
Pkg.activate(".")  # 現在のディレクトリの環境をアクティブ化
#Pkg.add("ITensors")  # ITensorsパッケージをインストール

  Activating project at `c:\Users\ito23\Documents\学院科目\物性物理学特論Ⅰ\Julia\day4\Mypackage`


In [2]:
using ITensors

In [3]:
# indexの定義
i = Index(2, "i")
j = Index(2, "j")
k = Index(2, "k")
l = Index(2, "l")

(dim=2|id=852|"l")

In [ ]:
# 4脚テンソルの生成
function tensor_4legs(K::Float64, i::Index, j::Index, k::Index, l::Index)
    W = ITensor(i, j, k, l)
    
    for ii in 0:1, jj in 0:1, kk in 0:1, ll in 0:1
        W[i => ii+1, j => jj+1, k => kk+1, l => ll+1] = exp(K * ((2 * ii - 1) * (2 * jj - 1) + (2 * jj - 1) * (2 * kk - 1) + (2 * kk - 1) * (2 * ll - 1) + (2 * ll - 1) * (2 * ii - 1)))
    end

    return W
end

J = 1.0
k_B = 1.0
T_C = 2 / log(1 + sqrt(2))
K = J / k_B / T_C
W = tensor_4legs(K, i, j, k, l)
println(T_C)
@show W

2.269185314213022
W = ITensor ord=4
Dim 1: (dim=2|id=671|"i")
Dim 2: (dim=2|id=736|"j")
Dim 3: (dim=2|id=419|"k")
Dim 4: (dim=2|id=852|"l")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2×2
[:, :, 1, 1] =
 5.828427124746189  1.0
 1.0                1.0

[:, :, 2, 1] =
 1.0                  1.0
 0.17157287525380993  1.0

[:, :, 1, 2] =
 1.0  0.17157287525380993
 1.0  1.0

[:, :, 2, 2] =
 1.0  1.0
 1.0  5.828427124746189


ITensor ord=4 (dim=2|id=671|"i") (dim=2|id=736|"j") (dim=2|id=419|"k") (dim=2|id=852|"l")
NDTensors.Dense{Float64, Vector{Float64}}

In [5]:
# 3脚テンソルの生成
function tensor_2legs(W::ITensor, l::Index)
    P = W * delta(l)
    
    return P
end

P = tensor_2legs(W, l)
@show P

P = ITensor ord=3
Dim 1: (dim=2|id=671|"i")
Dim 2: (dim=2|id=736|"j")
Dim 3: (dim=2|id=419|"k")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
 Inf   1.0
  2.0  2.0

[:, :, 2] =
 2.0   2.0
 1.0  Inf


ITensor ord=3 (dim=2|id=671|"i") (dim=2|id=736|"j") (dim=2|id=419|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [6]:
# 2脚テンソル生成
function tensor_2legs(P::ITensor, k::Index)
    C = P * delta(k)

    return C
end

C = tensor_2legs(P, k)
@show C

C = ITensor ord=2
Dim 1: (dim=2|id=671|"i")
Dim 2: (dim=2|id=736|"j")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 Inf    3.0
  3.0  Inf


ITensor ord=2 (dim=2|id=671|"i") (dim=2|id=736|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [7]:
# Pの拡大
function expand_tensor_3legs(W::ITensor, P::ITensor, i::Index, j::Index, k::Index, l::Index)
    inds_P = inds(P)
    
    p = Index(dim(inds_P[2]), "p")
    q = Index(dim(inds_P[3]), "q")

    inds_new_P = [l, p, q]

    P_b = replaceinds(P, inds_P, inds_new_P)

    P_a = W * P_b

    P_expanded = P_a * combiner(i, p; tags="x") * combiner(k, q; tags="y")

    return P_expanded
end

P_expanded = expand_tensor_3legs(W, P, i, j, k, l)
@show P_expanded

P_expanded = ITensor ord=3
Dim 1: (dim=2|id=736|"j")
Dim 2: (dim=4|id=426|"x")
Dim 3: (dim=4|id=386|"y")
NDTensors.Dense{Float64, Vector{Float64}}
 2×4×4
[:, :, 1] =
 Inf  Inf  Inf   3.0
 Inf  Inf   1.0  3.0

[:, :, 2] =
 Inf  NaN   3.0   2.0
 Inf   Inf  3.0  Inf

[:, :, 3] =
 Inf   3.0   Inf  Inf
  2.0  3.0  NaN   Inf

[:, :, 4] =
 3.0   1.0  Inf  Inf
 3.0  Inf   Inf  Inf


ITensor ord=3 (dim=2|id=736|"j") (dim=4|id=426|"x") (dim=4|id=386|"y")
NDTensors.Dense{Float64, Vector{Float64}}

In [8]:
# Cの拡大
function expand_tensor_2legs(W::ITensor, P::ITensor, C::ITensor, i::Index, j::Index, k::Index, l::Index)
    inds_P = inds(P)
    inds_C = inds(C)

    p = Index(dim(inds_P[2]), "p")
    q = Index(dim(inds_P[3]), "q")
    r = Index(dim(inds_P[2]), "r")
    s = Index(dim(inds_P[3]), "s")

    inds_new_P_L = [i, p, q]
    inds_new_P_R = [l, r, s]
    inds_new_C = [q, r]

    P_b_L = replaceinds(P, inds_P, inds_new_P_L)
    P_b_R = replaceinds(P, inds_P, inds_new_P_R)
    C_b = replaceinds(C, inds_C, inds_new_C)

    C_a = W * P_b_L * P_b_R * C_b

    C_expanded = C_a * combiner(j, p; tags="z") * combiner(k, s; tags="w")

    return C_expanded
end

C_expanded = expand_tensor_2legs(W, P, C, i, j, k, l)
@show C_expanded

C_expanded = ITensor ord=2
Dim 1: (dim=4|id=384|"z")
Dim 2: (dim=4|id=33|"w")
NDTensors.Dense{Float64, Vector{Float64}}
 4×4
  Inf   Inf   Inf   Inf
 NaN    Inf  NaN    Inf
  Inf  NaN    Inf  NaN
  Inf   Inf   Inf   Inf


ITensor ord=2 (dim=4|id=384|"z") (dim=4|id=33|"w")
NDTensors.Dense{Float64, Vector{Float64}}

In [9]:
# 圧縮
function compress_expansion(P::ITensor, C::ITensor, chi::Int64)
    x, y = inds(C)

    D, U = eigen(C, x, y)
    
    n = dim(x)

    p = Index(chi, "p")
    q = Index(chi, "q")

    C_compressed = ITensor(p, q)

    for ii in 1:chi, jj in 1:chi
        C_compressed[p => ii, q => jj] = D[inds(D)[1] => n - ii + 1, inds(D)[2] => n - jj + 1]
    end

    u = Index(dim(inds(U)[1]), "u")
    v = Index(chi, "v")
    
    U_compressed = ITensor(u, v)

    for ii in 1:n, jj in 1:chi
        U_compressed[u => ii, v => jj] = U[inds(U)[1] => n - ii + 1, inds(U)[2] => n - jj + 1]
    end

    inds_U_compressed = inds(U_compressed)
    inds_P = inds(P)

    a = Index(dim(inds(U_compressed)[1]), "a")
    b = Index(dim(inds(U_compressed)[2]), "b")
    c = Index(dim(inds(U_compressed)[1]), "c")
    d = Index(dim(inds(U_compressed)[2]), "d")
    e = Index(dim(inds(P)[1]), "e")

    inds_new_U_compressed_l = [a, b]
    inds_new_U_compressed_r = [c, d]
    inds_new_P = [e, a, c]

    U_compressed_l = replaceinds(U_compressed, inds_U_compressed, inds_new_U_compressed_l)
    U_compressed_r = replaceinds(U_compressed, inds_U_compressed, inds_new_U_compressed_r)
    P_b = replaceinds(P, inds_P, inds_new_P)

    P_compressed = P_b * U_compressed_l * U_compressed_r
    
    return C_compressed, P_compressed
end

# chi = 2
# C_compressed, P_compressed = compress_expansion(P_expanded, C_expanded, chi)
# @show C_compressed
# @show P_compressed




compress_expansion (generic function with 1 method)

In [11]:
# スピン相関の測定
function measure_spin_correlation(W::ITensor,P::ITensor, C::ITensor, i::Index, j::Index, k::Index, l::Index)
    inds_P = inds(P)
    inds_C = inds(C)

    a = Index(2, "a")
    b = Index(2, "b")
    g_la = Index(dim(inds_P[2]), "g_la")
    g_lb = Index(dim(inds_P[2]), "g_lb")
    g_al = Index(dim(inds_P[2]), "g_al")
    g_ar = Index(dim(inds_P[2]), "g_br")
    g_ra = Index(dim(inds_P[2]), "g_ra")
    g_rb = Index(dim(inds_P[2]), "g_rb")
    g_br = Index(dim(inds_P[2]), "g_br")
    g_bl = Index(dim(inds_P[2]), "g_bl")

    inds_new_P_l = [i, g_la, g_lb]
    inds_new_P_a = [j, g_ar, g_al]
    inds_new_P_r = [k, g_rb, g_ra]
    inds_new_P_b = [l, g_bl, g_br]
    inds_new_C_al = [g_al, g_la]
    inds_new_C_ar = [g_ra, g_ar]
    inds_new_C_br = [g_br, g_rb]
    inds_new_C_bl = [g_lb, g_bl]

    W_s = replaceinds(W, [i, l] => [a, b])
    P_l = replaceinds(P, inds_P, inds_new_P_l)
    P_a = replaceinds(P, inds_P, inds_new_P_a)
    P_r = replaceinds(P, inds_P, inds_new_P_r)
    P_b = replaceinds(P, inds_P, inds_new_P_b)
    C_al = replaceinds(C, inds_C, inds_new_C_al)
    C_ar = replaceinds(C, inds_C, inds_new_C_ar)
    C_br = replaceinds(C, inds_C, inds_new_C_br)
    C_bl = replaceinds(C, inds_C, inds_new_C_bl)
    
    S_l = ITensor(i, a)
    S_b = ITensor(l, b)
    
    for ii in 1:2
        S_l[i => ii, a => ii] = 2 * ii - 3
        S_b[l => ii, b => ii] = 2 * ii - 3
    end

    G = C_al * P_a * C_ar * P_r * C_br * P_b * C_bl * P_l

    O = G * W

    O_s = S_l * S_b * G * W_s

    SS_ev = O_s / O

    return only(SS_ev)
end

SS_ev = measure_spin_correlation(W, P_expanded, C_expanded, i, j, k, l)
println("Spin correlation: ", SS_ev)



Spin correlation: NaN


In [ ]:
# 自発磁化の測定
function measure_magnetization(P::ITensor, C::ITensor)
    inds_P = inds(P)
    inds_C = inds(C)

    a = Index(dim(inds_P[1]), "a")
    b = Index(dim(inds_P[1]), "b")
    q_la = Index(dim(inds_C[1]), "q_la")
    q_lb = Index(dim(inds_C[1]), "q_lb")
    q_ra = Index(dim(inds_C[1]), "q_ra")
    q_rb = Index(dim(inds_C[1]), "q_rb")
    q_a = Index(dim(inds_C[1]), "q_a")
    q_b = Index(dim(inds_C[1]), "q_b")

    inds_new_P_l = [a, q_la, q_lb]
    inds_new_P_r = [a, q_rb, q_ra]
    inds_new_P_r_s = [b, q_rb, q_ra]
    inds_new_C_al = [q_a, q_la]
    inds_new_C_ar = [q_ra, q_a]
    inds_new_C_br = [q_b, q_rb]
    inds_new_C_bl = [q_lb, q_b]

    P_l = replaceinds(P, inds_P, inds_new_P_l)
    P_r = replaceinds(P, inds_P, inds_new_P_r)
    P_r_s = replaceinds(P, inds_P, inds_new_P_r_s)
    C_al = replaceinds(C, inds_C, inds_new_C_al)
    C_ar = replaceinds(C, inds_C, inds_new_C_ar)
    C_br = replaceinds(C, inds_C, inds_new_C_br)
    C_bl = replaceinds(C, inds_C, inds_new_C_bl)

    S = ITensor(a, b)

    for ii in 1:2
        S[a => ii, b => ii] = 2 * ii - 3
    end

    Q = C_al * C_ar * P_r * C_br * C_bl * P_l

    Q_s = S * C_al * C_ar * P_r_s * C_br * C_bl * P_l

    S_ev = Q_s / Q

    return only(S_ev)
end

S_ev = measure_magnetization(P_expanded, C_expanded)
println("Magnetization: ", S_ev)

In [10]:
C_expanded = expand_tensor_2legs(W, P_expanded, C_expanded, i, j, k, l)
P_expanded = expand_tensor_3legs(W, P_expanded, i, j, k, l)
@show C_expanded
@show P_expanded

C_expanded = ITensor ord=2
Dim 1: (dim=8|id=641|"z")
Dim 2: (dim=8|id=240|"w")
NDTensors.Dense{Float64, Vector{Float64}}
 8×8
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
P_expanded = ITensor ord=3
Dim 1: (dim=2|id=736|"j")
Dim 2: (dim=8|id=733|"x")
Dim 3: (dim=8|id=725|"y")
NDTensors.Dense{Float64, Vector{Float64}}
 2×8×8
[:, :, 1] =
  Inf  Inf   Inf  Inf  Inf  Inf  Inf   6.0
 NaN   Inf  NaN   Inf  Inf  Inf   3.0  6.0

[:, :, 2] =
 Inf  NaN   Inf  NaN   Inf  NaN   6.0   3.0
 Inf   Inf  Inf   Inf  Inf   Inf  6.0  Inf

[:, :, 3] =
  Inf  Inf  NaN  NaN  Inf   6.0   Inf  Inf
 NaN   Inf  NaN  NaN   3.0  6.0  NaN   Inf

[:, :, 4] =
 Inf  NaN   NaN  NaN  6.0   3.0  Inf  Inf
 Inf   Inf  NaN  NaN  6.0  Inf   Inf  Inf

[:, :

ITensor ord=3 (dim=2|id=736|"j") (dim=8|id=733|"x") (dim=8|id=725|"y")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
chi = 100
C_compressed, P_compressed = compress_expansion(P_expanded, C_expanded, chi)
@show C_compressed
@show P_compressed